## Sarcasm Detector using Gaussian naive_bayes

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_json('Sarcasm_Headlines_Dataset_v2.json',lines=True)
df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [ ]:
print(df['is_sarcastic'].value_counts())
print(df.shape)
print(df['headline'][3].split(' '))
print(df['article_link'][3])

0    14985
1    13634
Name: is_sarcastic, dtype: int64
(28619, 3)
['inclement', 'weather', 'prevents', 'liar', 'from', 'getting', 'to', 'work']
https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031


In [ ]:
import nltk

from nltk.corpus import stopwords

nltk.download('stopwords');


In [ ]:
stop=stopwords.words('english')
print(stopwords.words('english'))


In [ ]:
from nltk.stem import PorterStemmer
sn=nltk.stem.SnowballStemmer('english')
print(sn.stem('scientists'))

In [ ]:
print(sn.stem('fishing'))
y=df['is_sarcastic']
df=df.drop('article_link',axis=1).drop('is_sarcastic',axis=1)



In [ ]:
def convert(l):
  str=''
  for ele in l:
    str=str+ele+" "

  return str

In [ ]:
def removestop(df):
    i=0
    list1=[]
    while i<len(df):
     l=df['headline'].iloc[i].split(' ') 
     for words in l:
        if words in stop:
          l.remove(words)
     s=convert(l)
     list1.append(s)
     i=i+1
    x=pd.DataFrame(list1,columns=['headline'])
    return x

df=removestop(df)

In [ ]:
def stemming(df):
  i=0
  l=[]
  l2=[]
  while i<len(df):
    l=df['headline'].iloc[i].split(' ')
    str=' '
    for words in l:
      str=str+sn.stem(words)+' '
    i=i+1
    l2.append(str)
  x1=pd.DataFrame(l2,columns=['headline']) 
  return x1 


In [ ]:
df=stemming(df)


In [ ]:
df.shape

(28619, 1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect=CountVectorizer(ngram_range=(1,1))
final_bi_gram=count_vect.fit_transform(df['headline'].values)

In [ ]:
print(final_bi_gram.shape)
x1=pd.DataFrame(final_bi_gram.todense())


(28619, 20023)


In [ ]:
def stoptest(s):
    l=s.split(' ')
    l1=[]
    for i in l:
      if i in stop:
        l.remove(i)
      else:
        l1.append(sn.stem(i))  
    
    return l1   

In [ ]:

df1=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
df1

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [ ]:
y1=df1['is_sarcastic']
df1=df1.drop('is_sarcastic',axis=1).drop('article_link',axis=1)


In [ ]:
df1=removestop(df1)
df1=stemming(df1)
df1

,headline
0,former versac store clerk sue secret black co...
1,"roseann reviv catch to thorni polit mood, bet..."
2,mom start fear son web seri closest thing wil...
3,"boehner want wife listen, come with altern de..."
4,j.k. rowl wish snape happi birthday the magic...
...,...
26704,american polit moral free-fal
26705,america best 20 hike
26706,repar obama
26707,isra ban target boycott support rais alarm ab...


In [ ]:
x1.shape

(28619, 20023)

In [ ]:
df1.shape

(26709, 1)

In [ ]:
y1.shape
#df1.shape
#print(df1.toarray().shape)

(26709,)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb=GaussianNB()
gnb.fit(x1,y)


GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
pred=[]
for i in range(0,26709):
  pred.append(gnb.predict(count_vect.transform(df1.iloc[i]).todense()))


In [ ]:
print(accuracy_score(y1,pred)*100)



85.92983638473922


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
neighbors=[10**-2,10**0,10**1,10**2,10**4]

cv_score=[]
for k in neighbors:
    
    clf=LogisticRegression(penalty='l2',C=1000,dual=True,n_jobs=-1,max_iter=10000,solver='liblinear')
    scores=cross_val_score(clf,x1,y,cv=10,scoring='accuracy')
    cv_score.append(scores.mean())
    print("THe value of k used is {}".format(k))

mse=[1-x for x in cv_score]

k1=neighbors[mse.index(min(mse))]
plt.plot(neighbors,mse)
plt.xlabel("C")
plt.ylabel("mse")
plt.show()

In [ ]:
pred=[]
for i in range(0,26709):
  pred.append(clf.predict(count_vect.transform(df1.iloc[i]).todense()))

 

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
print(sklearn.metrics.accuracy_score(y1,pred)*100) 
print(confusion_matrix(y1,pred))
print(f1_score(y1,pred))

In [ ]:
def stoptest(s):
    l=s.split(' ')
    print(l)

    for i in l:
      if i in stop:
        l.remove(i)

    return l 

In [ ]:
str=input('Enter the value of the string')

l=stoptest(str.lower())
print(l)
s=''
for i in l:
  s=s+sn.stem(i)+' '
print(s)


In [ ]:
l1=[]
l1.append(s)
print(l1)
x=clf.predict(count_vect.transform(l1).todense())


if x==0:
  print('not sarcastic')
else :
  print('sarcastic')

l=clf.predict_proba(count_vect.transform(l1).todense())
print('confidence percent',float(l[0][x]*100))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

tuned_parameters={'C':[10**-2,10**0,10**2]}
model=GridSearchCV(LogisticRegression(max_iter=100000,dual=True,n_jobs=-1,penalty='l2'),tuned_parameters,scoring='accuracy',cv=5)
model.fit(x1,y)

In [ ]:
pred=[]
for i in range(1,26709):
  pred.append(gnb.predict(count_vect.transform(df1.iloc[i]).todense()))

print(model.best_estimator)
print(model.score(pred,y1))